# 国立アイヌ民族博物館アイヌ語アーカイブ

In [16]:
from pathlib import Path
OUTPUT_SRC_DIR = Path('../output/src/ainu-archive')
if not OUTPUT_SRC_DIR.exists():
    OUTPUT_SRC_DIR.mkdir(parents=True)
OUTPUT_DIR = Path('../output/ainu-archive')
if not OUTPUT_DIR.exists():
    OUTPUT_DIR.mkdir(parents=True)

## Download the html files

In [17]:
# import requests
# for query in ['-', 'a', 'i', 'u', 'e', 'o']:
#     DIGITAL_ARCHIVE_URL = f"https://nam.aynu.org/dic?word={query}"
#     print(DIGITAL_ARCHIVE_URL)
#     response = requests.get(DIGITAL_ARCHIVE_URL)
#     response.raise_for_status()
#     with open(OUTPUT_SRC_DIR / f'{query}.html', 'w') as f:
#         f.write(response.text)

## Sanitize the html files


In [18]:
# import regex as re

# from tqdm.notebook import tqdm

# dd_regex = re.compile(r"(?s)(?<=<dd>)(.*?)(?=</dd>)")

# # Pattern for valid tags
# valid_tag_pattern = re.compile(
#     r"&lt;(/?[a-zA-Z][a-zA-Z0-9:-]*"
#     r'(?:\s+[a-zA-Z0-9:-]+(?:\s*=\s*(?:"[^"]*"|\'[^\']*\'|[^>\s]+))?)*'
#     r"\s*/?)&gt;"
# )

# if not (OUTPUT_DIR / "sanitized").exists():
#     (OUTPUT_DIR / "sanitized").mkdir(parents=True)

# def escape_all_angle_brackets_in_dd(html):
#     def replacer(match):
#         dd_content = match.group(1)
#         return dd_content.replace("<", "&lt;").replace(">", "&gt;")

#     return dd_regex.sub(replacer, html)


# def unescape_valid_tags_in_dd(html):
#     def unescape_tags(m):
#         return f"<{m.group(1)}>"

#     def dd_replacer(m):
#         dd_content = m.group(1)
#         # Replace &lt;validTag&gt; with <validTag>
#         dd_content_fixed = valid_tag_pattern.sub(unescape_tags, dd_content)
#         return dd_content_fixed

#     return dd_regex.sub(dd_replacer, html)


# def sanitize_dd_content(html):
#     # 1) Escape everything inside <dd>
#     step1 = escape_all_angle_brackets_in_dd(html)
#     # 2) Then unescape valid HTML tags inside <dd>
#     step2 = unescape_valid_tags_in_dd(step1)
#     return step2


# for file in tqdm(list(OUTPUT_SRC_DIR.glob("*.html"))):
#     with open(file, "r") as f:
#         html = f.read()

#         html = sanitize_dd_content(html)
#         with open(OUTPUT_DIR / 'sanitized' / file.with_suffix(".sanitized.html").name, "w") as f:
#             f.write(html)

In [19]:
import parsel
import regex as re
from tqdm.notebook import tqdm
from html import unescape

columns = ['lemma', 'def', 'pron', 'tag']
dictionary = set()

for file in tqdm(list((OUTPUT_DIR / 'sanitized').glob('*.sanitized.html'))):
    with open(file, 'r') as f:
        file_tag = file.stem
        
        # remove unnecessary spaces
        html = f.read().replace('\n', '')
        html = re.sub(r'\s+', ' ', html)
        
        selector = parsel.Selector(html)
        dl = selector.css('dl')
        if not dl:
            raise Exception(f'No dl element found in {file}')
            
        dts = dl.css('dt')
        dds = dl.css('dd')
        print(f'Total entries in {file}: {len(dts)}')
        
        for dt, dd in tqdm(list(zip(dts, dds))):
            # Extract the key and sound URL from the dt element
            lemma = (dt.xpath('string()').get() or '').strip()
            if dt.css('sup'):
                lemma = re.sub(r'\s*\d+\s*$', '', lemma)
            
            if lemma.startswith('‘') or lemma.startswith('“'):
                lemma = lemma.strip('‘“”’')
                
            sound_a = dt.css('a.sound')
            sound_url = sound_a.attrib['data-sound-url'] if sound_a else None
            
            # Extract the value from the dd element
            value = (dd.xpath('string()').get() or '').strip()

            value = re.sub(r'<span class="hl">', '', value)
            value = re.sub(r'</span>', '', value)

            value = unescape(value)
            
            dictionary.add((lemma, value, sound_url))

print("Total entries in all:", len(dictionary))


  0%|          | 0/6 [00:00<?, ?it/s]

Total entries in ../output/ainu-archive/sanitized/o.sanitized.html: 15885


  0%|          | 0/15885 [00:00<?, ?it/s]

Total entries in ../output/ainu-archive/sanitized/u.sanitized.html: 14903


  0%|          | 0/14903 [00:00<?, ?it/s]

...

  0%|          | 0/16814 [00:00<?, ?it/s]

Total entries in ../output/ainu-archive/sanitized/e.sanitized.html: 17800


  0%|          | 0/17800 [00:00<?, ?it/s]

Total entries in ../output/ainu-archive/sanitized/-.sanitized.html: 19047


  0%|          | 0/19047 [00:00<?, ?it/s]

In [3]:
for i in sorted(dictionary, key=lambda x: len(x[1]), reverse=True)[:10]:
    print(i)


('wa', 'ワ 【接助】…して(二つの動詞句をつないで一まとまりのことがらとしてまとめる働きをする。 具体的には､ 次のようないろいろな場合に使われる)。 ①(一つのもの/ことについて二つ以上の説明や叙述をする場合にその二つの動詞句をつなぐ｡) upakno oka wa sattek kusu oka wa ウパｸノ オカ ワ サッテｸ クス オカ ワ 同じぐらいの大きさでやせこけていて｡(Ｓ会話) Kusur un oruspe ne wa teeta oruspe ne wa a=ewkoysoytak kor síran pe tasi an nek! クスルン オルｼペ ネ ワ テエタ オルｼペ ネ ワ アエウコイソイタｸ コﾛ シラン ペ タシ アン ネｸ! (これは)釧路の話で昔の話で､ 私たちはそれについて話し合っているのですよ｡(Ｓ会話) sunke ka eaykap wa anpe patek ye rusuy pe スンケ カ エアイカﾌﾟ ワ アンペ パテｸ イェ ルスイ ペ うそをつくことができなくて本当のことばかり言うもの｡(Ｓ会話) ②(前の動詞句が後の動詞句の表す出来事の方法や意図などを説明する｡) ku=hoyupu wa k=ek クホユプ ワ ケｸ (私は)走って来た。 hosippa wa paye ホシッパ ワ パイェ (彼らは)もどって行った。 ③(前の動詞句の出来事が終わってから後の動詞句の出来事が起こることを言う｡)…して。 otcike huraye wa pirpa オッチケ フライェ ワ ピﾘパ お膳を洗ってふきなさい｡(Ｗ) núman k=ek wa tanto suy ukoytak=an hawe ne ヌマン ケｸ ワ タント スイ ウコイタｸアン ハウェ ネ 私は昨日来て､ 今日また(姉と私と)二人で会話をするのです｡(Ｓ会話) ne to or ta cip hene a=kar wa oro ta a=anú wa ネ ト オッタ チﾌﾟ ヘネ アカﾗ ワ オロ タ アアヌ ワ その湖に舟でもつくってそこに置いて｡(Ｓ会話) ④(前の出来事が後の出来事の原因か理由になっている｡)…して。 k=úkao oyra wa rurikan クカオ オイラ ワ ルリカン (私は洗濯物を)しまい忘れて

In [4]:
for i in dictionary:
    if not i[0]:
        print(i)

('', 'フナｸ タ エパｸ/フナｸ タ エパッカ/フナｸ タ エパーカ 【慣用句】ちょうどいいあんばいに(…してよかった｡) hunak ta epak e=ek wa ku=mismu ka somo ki フナｸ タ エパｸ エエｸ ワ クミｼム カ ソモ キ いいあんばいに来てくれたので私はさびしくない｡(Ｓ) hunak ta epak ka e=pa p an un フナｸ タ エパｸ カ エパﾌﾟ アヌン いいあんばいにあなたが見つけたね(他の人だったら見つけても拾ってくれないだろうが)｡(Ｓ) hunak ta epaː ka somo kú=itura wa ene kú=iki hi an sekor ku=raman, poro k=éyayomusu フナｸ タ エパー カ ソモ クイトゥラ ワ エネ クイキ ヒ アン セコﾛ クラマン､ ポロ ケヤヨムス (ついていってこんなに何度もトイレに起きたら他の人も眠れないし自分も気の毒に思うところだった､ ) いいあんばいについて行かないでこうしているのだと思ったらほんとに行かないでよかったなあと思った｡(Ｓ) {E: fortunately.} （出典：田村、方言：沙流）', None)
('', '『エムン』 §278 オカヒジキ (3) 植物誌(E、p.380)には、なお、『エムン』(北見アイヌ名)ともある。e-mun“食う・草”の義か。 （出典：知里植物編、方言：）', None)
('', '『ヲイエンキキナ』 §278 オカヒジキ (1) 概報(G、p.33)に、Oienki-kina『ヲイエンキキナ』とある。明らかに「あイ・エンケ・キナ」áy-enke-kina“とげ・するどい・草”の誤写である。それが、植物誌(E、p.380)にもそのまま再録され、辞書もその誤りをそのまま踏襲している。 （出典：知里植物編、方言：）', None)
('', '『トーカリ』 §470 マリモ 毬藻 (1) コタン生物記(p.77)に『トーカリ』とある。これは「と・カリﾌﾟ」tó-karip“沼の・球”ということである。to(沼)、kari(回る)、p(もの)。 （出典：知里植物編、方言：）', None)
('', '『エバー』 §054 キムラタケ オニク (2) 『エバー』 ｟B、E、F｠ （

In [5]:
import csv
import unicodedata
with open(OUTPUT_DIR / 'dictionary.tsv', 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(['lemma', 'definition', 'sound_url'])
    for (lemma, definition, sound_url) in sorted(dictionary, key=lambda x: unicodedata.normalize('NFKC', x[0]).lower()):
        if lemma == '':
            continue

        writer.writerow((lemma, definition, sound_url))

In [6]:
for i in dictionary:
    if not re.search(r'（出典：.*?、方言：.*?）$', i[1]):
        print(i)

In [7]:
# import pandas as pd
# import regex as re
# df = pd.read_csv(OUTPUT_DIR / "dictionary.tsv", sep='\t').dropna(subset=["lemma"])
# # df[df['def'].apply(lambda x: re.search(r'^[\p{Katakana}\-]', x) is None)].to_csv('./result.csv')
# df[
#     df["definition"].apply(lambda x: re.search(r"（出典：.*?、方言：.*?）$", x) is None)
# ].to_csv(OUTPUT_DIR / "dictionary_categorized.tsv", sep="\t", index=False)

In [80]:
# count all words in the dictionary in pos column   
# df['pos'].value_counts()

# for i in dictionary:
#     if re.search(r'（出典：.*?、方言：.*?）$', i[1]):
#         print(i)


### Clean up the dictionary

In [8]:
import pandas as pd
from pathlib import Path

df_archive = (
    pd.read_csv(OUTPUT_DIR / "dictionary.tsv", sep='\t')
    .dropna(subset=["lemma"])
    .drop_duplicates(subset=["lemma", "definition"])
    .reset_index(drop=True)
)

HALF_WIDTH_KANA_2_SMALL_KANA = {
    "ｸ": "ㇰ",
    "ｼ": "ㇱ",
    "ｽ": "ㇲ",
    "ﾄ": "ㇳ",
    "ﾇ": "ㇴ",
    "ﾊ": "ㇵ",
    "ﾋ": "ㇶ",
    "ﾌﾟ": "ㇷ゚",
    "ﾌ": "ㇷ",
    "ﾍ": "ㇸ",
    "ﾎ": "ㇹ",
    "ﾑ": "ㇺ",
    "ﾗ": "ㇻ",
    "ﾘ": "ㇼ",
    "ﾙ": "ㇽ",
    "ﾚ": "ㇾ",
    "ﾛ": "ㇿ",
}


# Replace half-width katakana with small katakana
df_archive["definition"] = df_archive["definition"].apply(
    lambda x: re.sub(
        rf'({"|".join(HALF_WIDTH_KANA_2_SMALL_KANA.keys())})',
        lambda m: HALF_WIDTH_KANA_2_SMALL_KANA[m.group(1)],
        x,
    )
)

In [9]:
df_archive[["lemma", "definition"]].to_csv(
    OUTPUT_DIR / "cleaned-dictionary.tsv", sep="\t", index=False
)

In [10]:
import pandas as pd
from pathlib import Path
cleaned_dictionary = pd.read_csv(filepath_or_buffer=OUTPUT_DIR / 'cleaned-dictionary.tsv', sep='\t')

print(len(cleaned_dictionary))

pd.set_option('display.max_colwidth',1000)
cleaned_dictionary.head()

25646


,lemma,definition
0,#NAME?,ターラ 【tara】〜に，モイレターラ＝遅くに．ラッチターラ＝静かに． （出典：萱野、方言：沙流）
1,#NAME?,アㇱ 【=as】私，私たち〔人称接辞〕． （出典：萱野、方言：沙流）
2,#NAME?,アン 【=an】私，私たち〔人称接辞〕． （出典：萱野、方言：沙流）
3,(a-)ekonramu-sitne,"エコンラムシッネ §245．きぜつする（気絶する）；気が遠くなる；気を失う；人事不省になる(13)うつらうつらする[気を失ってから我に返る直前の半意識の状態にある](a-)ekonramu-sitne〔e-kón-ra-mu-šit-ne エこンラム･シッネ〕[a-(我)、e(それについて)、kon(<kor 持っている)、ramu(その心･心臓〔が〕)、sit(糸などのもつれた玉)、ne(のようになる)]｟ホロベツ,サル―ユ研Ⅱ, pp.466〜467｠ （出典：知里人間編I、方言：）"
4,(a-)ekonramu-tanak-tanak,エコンラムタナㇰタナㇰ §245．きぜつする（気絶する）；気が遠くなる；気を失う；人事不省になる(12)うつらうつらする[気を失ってから我に返ろうとする前の半意識の状態、ゆめうつつの状態にある](a-)ekonramu-tanak-tanak〔エこンラム･たナㇰタナㇰ〕[a-(我)、e(それについて)、kon(<kor もっている、自分の)、ramu(その心〔が〕]、tanak-tanak(うつらうつらする)>｟サル―ユ研Ⅱ、 p.599｠ （出典：知里人間編I、方言：）


In [11]:
# Find all 田村's in the dictionary (ending with 田村)

tamura_entries = cleaned_dictionary.loc[cleaned_dictionary['definition'].str.contains('（出典：田村、方言：沙流）')]

# Remove the citation from the definition
tamura_entries.loc[:, 'definition'] = tamura_entries['definition'].apply(lambda x: re.sub(r'（出典：田村、方言：沙流）', '', x))
tamura_entries

,lemma,definition
5,-a,ア 【接尾】[名詞の所属形形成](所属語尾とも呼ぶ。 母音 a ア + 子音に終る語幹のうちのあるものにつく。 この母音がついて所属形の｢短い形｣ができる。 さらにその後に ha ハ がついて(つまり語幹に -aha アハ がついて)所属形の｢長い形｣がつくられる。 本辞典では長短二つの形を合わせて…a(ha) のように表記してある｡) sar サㇻ [概]尾；sara(ha) サラ(ハ) [所]…の尾。
6,-aha,アハ 【接尾】[名詞の所属形形成](｢所属語尾｣とも呼ぶ。 所属形の｢長い形｣をつくる｡) ☞-a ア
7,-ar,アㇻ 【接尾】[不定使役形接尾辞](＝-yar ヤㇻ)(子音の後で y が落ちた形。 母音の後では -yar ヤㇻ の形が使われる｡) kar カㇻ …をつくる；karar カラㇻ …を人につくってもらう。
8,-asnu,アㇱヌ 【接尾】(語根またはそれから派生した語基に接尾して､ 性質を表す自動詞(形容詞)をつくる｡)…が優れている。 kam カㇺ 肉；kamasnu カマㇱヌ 色白できれいな肌である。 tum トゥム 力；tumasnu トゥマㇱヌ 力が強い。 mon モン[語根]手；monasnu モナㇱヌ 手が早い､ 仕事がはかどる(monasap モナサㇷ゚ は《手がのろい､ 仕事がはかどらない》)｡
9,-atki,アッキ 【接尾】(自動詞をつくる。 限られた決まった語にのみ現われる｡)tusus トゥスㇱ (擬態)；tususatki トゥスサッキ ガタガタふるえる。 sam サㇺ 横；samatki サマッキ 横向きになる。 sine シネ 一つの；sineatki シネアッキ 決まる。 sikari シカリ 丸い；sikannatki シカンナッキ 回る。
...,...,...
25641,útek,ウテㇰ 【他動】(人)を使いにやる､ (人)を(使い走りの仕事に)使う。 póho útek hawe ne ポホ ウテㇰ ハウェ ネ (その母親は)息子を使いにやったんだね｡(Ｓ会話) cútek ussiw チュテㇰ ウッシウ 使い走り(口上を持って他の人のところへ使いに行くなど)のための使用人。 {E: to use (someone) as a work helper} ｟テープ｠
25642,útur,ウトゥㇽ 【位名】[概](所は útur(u)ke(he) ウトゥル/ウトゥㇽ/ウトゥルケ(ヘ)/ウトゥㇽケ(ヘ))…の下座､ 木尻座､ (家の中の)出入口に近い方､ 東窓から遠い方､ (家の外でも)西側。 ☆参考 utur ウトゥㇽ ｢間｣とは別の語である。 アクセントが違う。 cise útur kus チセ ウトゥㇽ クㇱ 家の西側を通る(cise útur péka kus チセ ウトゥㇽ ペカ クㇱ とも言う)｡(Ｓ) ☆参考 所属形 úturu ウトゥル の形は未出。 {E: the lower seat of (a house etc.); the western side of (a house etc.).} ｟テープ｠
25643,útur(u)ke(he),"ウトゥㇽケ(ヘ)/ウトゥルケ(ヘ) 【位名】[所](概は útur ウトゥㇽ)[utur-u-ke …の下座・(所属語尾)・(よりはっきりさせる所属語尾)…の所] その下座(その西側/入口側)の所。 osisoun kamuy ekasi a wa an, úturuke ta kamuy ne noyne an húci a wa an オシソウン カムイ エカシ ア ワ アン､ ウトゥルケ タ カムイ ネ ノイネ アン フチ ア ワ アン 右座(いろりの北側､ 主人の席)に､ 神なる老人が座っていた､ その下座側(西側､ 入口側)に､ 神のような老女が座っていた｡(Ｗ神謡語り) ☆参考 úturu ウトゥル の形は未出。 {E: the lower seat of…} ｟テープ｠"
25644,úturke,ウトゥㇽケ 【位名】☞útur(u)ke(he) ウトゥㇽケ(ヘ)/ウトゥルケ(ヘ)


In [12]:
# Create a copy to avoid SettingWithCopyWarning
tamura_entries_copy = tamura_entries.copy()

# Extract transliteration and definition using .loc
tamura_entries_copy.loc[:, 'translit'] = tamura_entries_copy['definition'].apply(
    lambda x: re.match(r'(.*?)(【.*)', x).group(1) if re.match(r'(.*?)(【.*)', x) else re.match(r'(\p{scx=Katakana}+)', x).group(1) if re.match(r'(\p{scx=Katakana}+)', x) else None
)

tamura_entries_copy.loc[:, 'definition'] = tamura_entries_copy['definition'].apply(
    lambda x: re.match(r'(.*?)(【.*)', x).group(2) if re.match(r'(.*?)(【.*)', x) else re.match(r'(\p{scx=Katakana}+)', x).group(1) if re.match(r'(\p{scx=Katakana}+)', x) else None
)

# Reassign to original variable with selected columns
tamura_entries_parsed = tamura_entries_copy[['lemma', 'translit', 'definition']]

In [13]:
tamura_entries_parsed.to_csv(OUTPUT_DIR / "tamura-entries.tsv", sep="\t", index=False)

In [14]:
import pandas as pd
tamura_entries_parsed = pd.read_csv(OUTPUT_DIR / "tamura-entries.tsv", sep="\t")
tamura_entries_parsed

,lemma,translit,definition
0,-a,ア,【接尾】[名詞の所属形形成](所属語尾とも呼ぶ。 母音 a ア + 子音に終る語幹のうちのあるものにつく。 この母音がついて所属形の｢短い形｣ができる。 さらにその後に ha ハ がついて(つまり語幹に -aha アハ がついて)所属形の｢長い形｣がつくられる。 本辞典では長短二つの形を合わせて…a(ha) のように表記してある｡) sar サㇻ [概]尾；sara(ha) サラ(ハ) [所]…の尾。
1,-aha,アハ,【接尾】[名詞の所属形形成](｢所属語尾｣とも呼ぶ。 所属形の｢長い形｣をつくる｡) ☞-a ア
2,-ar,アㇻ,【接尾】[不定使役形接尾辞](＝-yar ヤㇻ)(子音の後で y が落ちた形。 母音の後では -yar ヤㇻ の形が使われる｡) kar カㇻ …をつくる；karar カラㇻ …を人につくってもらう。
3,-asnu,アㇱヌ,【接尾】(語根またはそれから派生した語基に接尾して､ 性質を表す自動詞(形容詞)をつくる｡)…が優れている。 kam カㇺ 肉；kamasnu カマㇱヌ 色白できれいな肌である。 tum トゥム 力；tumasnu トゥマㇱヌ 力が強い。 mon モン[語根]手；monasnu モナㇱヌ 手が早い､ 仕事がはかどる(monasap モナサㇷ゚ は《手がのろい､ 仕事がはかどらない》)｡
4,-atki,アッキ,【接尾】(自動詞をつくる。 限られた決まった語にのみ現われる｡)tusus トゥスㇱ (擬態)；tususatki トゥスサッキ ガタガタふるえる。 sam サㇺ 横；samatki サマッキ 横向きになる。 sine シネ 一つの；sineatki シネアッキ 決まる。 sikari シカリ 丸い；sikannatki シカンナッキ 回る。
...,...,...,...
9357,útek,ウテㇰ,【他動】(人)を使いにやる､ (人)を(使い走りの仕事に)使う。 póho útek hawe ne ポホ ウテㇰ ハウェ ネ (その母親は)息子を使いにやったんだね｡(Ｓ会話) cútek ussiw チュテㇰ ウッシウ 使い走り(口上を持って他の人のところへ使いに行くなど)のための使用人。 {E: to use (someone) as a work helper} ｟テープ｠
9358,útur,ウトゥㇽ,【位名】[概](所は útur(u)ke(he) ウトゥル/ウトゥㇽ/ウトゥルケ(ヘ)/ウトゥㇽケ(ヘ))…の下座､ 木尻座､ (家の中の)出入口に近い方､ 東窓から遠い方､ (家の外でも)西側。 ☆参考 utur ウトゥㇽ ｢間｣とは別の語である。 アクセントが違う。 cise útur kus チセ ウトゥㇽ クㇱ 家の西側を通る(cise útur péka kus チセ ウトゥㇽ ペカ クㇱ とも言う)｡(Ｓ) ☆参考 所属形 úturu ウトゥル の形は未出。 {E: the lower seat of (a house etc.); the western side of (a house etc.).} ｟テープ｠
9359,útur(u)ke(he),ウトゥㇽケ(ヘ)/ウトゥルケ(ヘ),"【位名】[所](概は útur ウトゥㇽ)[utur-u-ke …の下座・(所属語尾)・(よりはっきりさせる所属語尾)…の所] その下座(その西側/入口側)の所。 osisoun kamuy ekasi a wa an, úturuke ta kamuy ne noyne an húci a wa an オシソウン カムイ エカシ ア ワ アン､ ウトゥルケ タ カムイ ネ ノイネ アン フチ ア ワ アン 右座(いろりの北側､ 主人の席)に､ 神なる老人が座っていた､ その下座側(西側､ 入口側)に､ 神のような老女が座っていた｡(Ｗ神謡語り) ☆参考 úturu ウトゥル の形は未出。 {E: the lower seat of…} ｟テープ｠"
9360,úturke,ウトゥㇽケ,【位名】☞útur(u)ke(he) ウトゥㇽケ(ヘ)/ウトゥルケ(ヘ)


In [15]:
from typing import TypedDict
import regex as re
class TamuraEntry(TypedDict):
    lemma: str
    pos: str
    definition: str

class ParsedTamuraEntry(TypedDict):
    lemma: str
    poses: list[str]
    glosses: list[str]

entries: list[ParsedTamuraEntry] = []

for i, entry in enumerate(tamura_entries.to_dict(orient='records')):
    pos = (match := re.search(r'【(?:< )?(.*?)】', entry['definition'])) and match.group(1)

    if not pos:
        # print(entry['definition'])
        continue

    if pos[0].isascii():
        print(len(entry['definition']))
        # print(entry['def'])
        break

    pos_map = {
        "接尾": "sfx",
        "接頭": "pfx",
        "副助詞": "advp",
        "接続助詞": "sconj",
        "疑問": "int",
        "形式名詞": "nmlz",
        "動詞": "v",
        "自動": "vi",
        "他動": "vt",
        "複他動": "vd",
        "完動": "vc",
        "複他": "vd",
        "名": "n",
        "名詞": "n",
        "連体詞": "adn",
        # "形容詞": "adj",
        "副詞": "adv",
        "接続詞": "cconj",
        "助詞": "post",
        "助動": "auxv",
        "副": "adv",
        "終助": "sfp",
        "接尾辞": "sfx",
        "接頭辞": "pfx",
        "間投": "intj",
        "後置副詞": "padv",
        "人接": "pers",
        "繋辞": "cop",
        "位名": "nl",
        # "複数形": "pl",
        "複数形": "v",
        "副助": "advp",
        "接助": "sconj",
        "疑問詞": "int",
        "形名": "nmlz",
        "他動語幹": "root",
        "自動語幹": "root",
        "他動語根": "root",
        "語構成要素": "root",
        "語根": "root",
        "名詞語根": "root",
        "代名": "pron",
        "連他動": "colloc",
        "後副": "padv",
        # "接": "conj",
        "接": "cconj",
        "連体": "adn",
        "格助": "postp",
        "助": "parti",
        "分離他動": "vt",
        "慣用句": "idiom",
        "数名": "n",
    }

    pos = re.sub(r"\(?[?？]\)?", "", pos)
    poses = re.split(r"[＋/+(]", pos)
    for pos in poses:
        if pos not in pos_map:
            print(pos)
            continue

    parsed_poses = [pos_map.get(pose, pose) for pose in poses]

    definition = re.sub(r'【(.*?)】', '', entry['definition'])
    definition = re.sub(r'（出典：.*?）', '', definition)
    definition = re.sub(r'\[.*?\]', '', definition)
    definition = re.sub(r'\(.*\)', '', definition)
    definition = re.sub(r"[☞☆].*", "", definition)
    definition = re.sub(r'\{.*?\}', '', definition)
    definition = definition.replace("｟テープ｠", "")

    glosses = re.split(r"[\p{P}①②③④⑤⑥⑦⑧⑨⑩]", definition)
    glosses = [gloss.strip() for gloss in glosses if gloss.strip()]
    glosses = [gloss for gloss in glosses if not gloss[0].isascii()]

    entries.append({
        "lemma": entry['lemma'],
        "poses": parsed_poses,
        "glosses": glosses
    })

82


In [151]:
import json
with open(OUTPUT_DIR / "tamura-entries-parsed.json", "w") as f:
    json.dump(entries, f, ensure_ascii=False)